In [1]:
!nvidia-smi

Mon Nov  2 15:33:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
pip install transformers

     |████████████████████████████████| 1.3MB 12.3MB/s 
     |████████████████████████████████| 1.1MB 57.7MB/s 
     |████████████████████████████████| 890kB 56.4MB/s 
     |████████████████████████████████| 2.9MB 49.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=e2c2d507cc4964f4d5925c1041c2202fc578263281b37e383817bbfed2399238
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import torch
from transformers import BertTokenizer, BertModel

In [4]:
def dataset(filename):
    with open(filename,'r') as fp:
        lines = [line.strip() for line in fp]
    return lines

In [5]:
def wordData(data):
    wordLines = data
    words = []
    probabilities = []
    wordList = []
    pos = []
    empty = []
    for line in wordLines:
        lineSplit = line.strip().split('\t')
        if line:
            word = lineSplit[1]
            prob = lineSplit[4]
            temp = lineSplit[5]
            words.append(word)
            probabilities.append(prob)
            pos.append(temp)
        elif not (len(empty) and []):
            wordList.append((words, pos, probabilities))
            words = []
            probabilities = []
            pos = []
    return wordList

In [6]:
def wordtest(data):
    wordLines = data
    words = []
    testWord = []
    empty = []
    for line in wordLines:
        lineSplit = line.strip().split('\t')
        if line:
            word = lineSplit[1]            
            words.append(word)
        elif not len(empty):
            testWord.append(words)
            words = []       
    return testWord

In [7]:
def preTokenizing(data):
    t = []
    po = []
    prob = []
    for i,j,k in data:
        text = []
        pos = []
        probs = []
        for l in i:
            text.append(l)
        for m in j:
            pos.append(m)
        for n in k:
            probs.append(n)
        t.append(text)
        po.append(pos)
        prob.append(probs)
    return t,po, prob

In [8]:
TRAINING_FILE = "train.txt"
DEV_FILE = "dev.txt"
TEST_FILE = "test_data.txt"

In [9]:
trainText = wordData(dataset(TRAINING_FILE))
testEval = wordtest(dataset(TEST_FILE))
devText = wordData(dataset(DEV_FILE))


trainWords,trainTags, trainLabels = preTokenizing(trainText)
devWords, devTags, devLabels = preTokenizing(devText)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_text = []
for i in trainWords:
    sent = ""
    for j in i:
        if sent == "":
            sent += j
        else:
            sent = sent + " " + j
    tokens = tokenizer.tokenize(sent)
    token_ids = tokenizer.encode_plus(tokens, add_special_tokens=False, return_attention_mask=False, return_tensors='pt')
    tokenized_text.append(token_ids)


In [10]:
print(tokenized_text[0])

{'input_ids': tensor([[3828, 1996, 3058]]), 'token_type_ids': tensor([[0, 0, 0]])}


In [11]:
import torch
import torch.nn as nn

In [12]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.linear = nn.Linear(768,3)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens):
        _, dropout_output = self.bert(tokens)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba
    

In [14]:
bert_clf = BertBinaryClassifier()
optimizer = torch.optim.Adam(bert_clf.parameters(), lr=3e-6)
bert_clf.train()
for epoch_num in range(1):
    for batch_data,batch_prob in zip(tokenized_text,trainLabels):
        probas = bert_clf(batch_data['input_ids'])
        print(probas)
        loss_func = nn.MSELoss()
        batch_loss = loss_func(probas, torch.FloatTensor(batch_prob))
        bert_clf.zero_grad()
        batch_loss.backward()
        optimizer.step()


tensor([[0.4371, 0.4922, 0.5090]], grad_fn=<SigmoidBackward>)


ValueError: ignored